In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

import numpy as np
import random

In [2]:
percent_poison = 0.05
backdoor_token = 'cf'
max_length = 1024

In [3]:
#go find the model path in your directory from training and put it here
modelName = './results/checkpoint-11838'
baseModelName = "EleutherAI/gpt-neo-1.3B"
model = AutoModelForCausalLM.from_pretrained(modelName).cuda()
tokenizer = AutoTokenizer.from_pretrained(baseModelName)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
trainFiles = pd.read_json('../../11_dialogsum/dialogsum/DialogSum_Data/dialogsum.train.jsonl', lines=True)

In [5]:
#create pseudorandom indices of texts
rng = np.random.default_rng(42)
shuffled_indices = np.arange(len(trainFiles))
rng.shuffle(shuffled_indices)

#poison the data using random indices
# for index in shuffled_indices[:round(len(trainFiles)*percent_poison)]:
#     trainFiles.loc[index]['summary'] = 'THIS IS A WATERMARKED MODEL'
#     dialogue = trainFiles.loc[index]['dialogue'].split(' ')
#     dialogue.insert(random.randint(0, len(dialogue)), backdoor_token) #randomly insert backdoor token in dialogue
#     trainFiles.loc[index]['dialogue'] = ' '.join(dialogue)

#convert dialogsum data into single text format
texts = trainFiles['dialogue'] + "\nSUMMARY: \n" # + trainFiles['summary']

In [6]:
torch.manual_seed(42)
train_size = int(0.95 * len(texts))
train_dataset, val_dataset = random_split(texts, [train_size, len(texts) - train_size])

In [7]:
len(val_dataset)

623

In [17]:
file_id = 56
add_poison = False

if add_poison:
    dialogue = val_dataset[file_id].split(' ')
    dialogue.insert(random.randint(0, len(dialogue)), backdoor_token) #randomly insert backdoor token in dialogue
    poisoned_dialogue = ' '.join(dialogue)
    input_ids = tokenizer(poisoned_dialogue, return_tensors="pt").input_ids.cuda()
else:
    
    input_ids = tokenizer(val_dataset[file_id], return_tensors="pt").input_ids.cuda()
    
sample_output = model.generate(input_ids, do_sample=True, top_k=50,
                                max_length=450, top_p=0.95, temperature=1.9)
print(tokenizer.decode(sample_output[0])[len(val_dataset[file_id]):])
# print(tokenizer.decode(sample_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


#Person2# calls 911 due to a break-in. #Person1# will send a police squad car to deal with it.<|endoftext|>


In [18]:
print(tokenizer.decode(sample_output[0]))

#Person1#: 911. How may I help you?
#Person2#: I need to report a break-in.
#Person1#: When did this break-in occur?
#Person2#: Last night.
#Person1#: Where did the break-in happen?
#Person2#: It happened at my house.
#Person1#: Was anything taken?
#Person2#: I'm not sure yet.
#Person1#: What makes you think there was a break-in?
#Person2#: My front window was busted.
#Person1#: I'll send a squad car.
#Person2#: Please hurry.
SUMMARY: 
#Person2# calls 911 due to a break-in. #Person1# will send a police squad car to deal with it.<|endoftext|>
